Todo

- Exportovat seznam seznamových sloupců, aby ho další sešit v pipelině načítal automaticky.

- Ještě mnoho sloupců seznamových, kvůli čárkám ", sro" nutno upravit při scrapování.

- Ukazují se exulantské filmy! Zároveň tam ale chceme nechat koprodukce. (Ten samý problém u herectva.)

- Premiéra do datetime.

- Rok výroby do dvou sloupců.

In [1]:
import pandas as pd

In [2]:
df = pd.read_json("filmy_raw.json")

In [3]:
df.columns.to_list()

['Film',
 'Země původu',
 'Copyright',
 'Rok výroby',
 'Premiéra',
 'Minutáž',
 'Režie',
 'Kategorie',
 'Žánr',
 'Typologie',
 'Originální název',
 'Český název',
 'Anglický název',
 'Souběžný název',
 'Pracovní název',
 'Pomocná režie',
 'Asistent režie',
 'Původní filmový námět',
 'Scénář',
 'Dramaturg',
 'Kamera',
 'Asistent kamery',
 'Architekt',
 'Návrhy kostýmů',
 'Masky',
 'Střih',
 'Zvuk',
 'Producent',
 'Vedoucí výroby',
 'Zástupce vedoucího výroby',
 'Odborný poradce',
 'Hudba',
 'Choreografie',
 'Lokace',
 'Distribuční slogan',
 'Výrobce',
 'S podporou',
 'Nositelé copyrightu',
 'Distribuce',
 'Délka',
 'Distribuční nosič',
 'Poměr stran',
 'Barva',
 'Zvukový systém/formát',
 'Verze',
 'Mluveno',
 'Podtitulky',
 'Úvodní/závěrečné titulky',
 'Anotace',
 'Obsah',
 'Technický scénář',
 'Výtvarník',
 'Animace',
 'Triky',
 'Postprodukce',
 'Návrhy titulků',
 'Pedagogické vedení',
 'Použitá hudba',
 'Studentský film',
 'Původní metráž',
 'Mezititulky',
 'Ocenění',
 'Vizuální efekt

In [4]:
def citlivy_orez(x):
    if isinstance(x, str):
        x = x.strip()
    if isinstance(x, list):
        try:
            x = [y.strip() for y in x]
        except:
            pass
    return(x)

In [5]:
def do_seznamu(polozka):
    try:
        if "," in polozka:
            polozka = str(polozka).split(",")
            polozka = [p.strip() for p in polozka]
    except:
        pass
    return(polozka)

In [6]:
df = df.apply(lambda x: citlivy_orez(x))

In [7]:
seznamove_sloupce = ["Žánr",
                     "Režie",
                     "Scénář",
                     "Mluveno",
                     "Barva",
                     "Zvukový systém/formát",
                     "Země původu",
                     "Úvodní/závěrečné titulky",
                     "Distribuční nosič",
                     "Země původu",
                     "Typologie"]

In [8]:
for s in seznamove_sloupce:
    df[s] = df[s].apply(lambda x: do_seznamu(x))

In [9]:
df["Minutáž"] = df["Minutáž"].apply(lambda x: str(x).replace(" min",""))
df["Minutáž"] = pd.to_numeric(df["Minutáž"], errors="coerce")

In [10]:
def seznam_lokaci(lokace_hrube):
    lokace_ciste = []
    if lokace_hrube:
        lokace_hrube = lokace_hrube.split("),")
        for x in lokace_hrube:
            if "(" in x:
                x = x + ")"
            x = x.strip()
            x = x.replace("))",")")
            if x[0:2] == ", ":
                x = x[2:]
            if x != ")":
                lokace_ciste.append(x)
    return(lokace_ciste)

In [11]:
df["Lokace"] = df["Lokace"].apply(lambda x: seznam_lokaci(x))

In [12]:
def ratio(x):
    if x:
        x = x.replace("1:","")
        x = x.split(", ")
        x = [y.strip() for y in x]
        x = [y.replace(",",".") for y in x]
        x = [pd.to_numeric(y) for y in x]
        if len(x) == 1:
            x = x[0]
    return(x)

In [13]:
df["Poměr stran"] = df["Poměr stran"].apply(lambda x: ratio(x))

In [40]:
df["Ocenění"] = df["Ocenění"].apply(lambda x: kill_whitespace(x))

In [41]:
df = df.sort_values(by = "Copyright")

In [42]:
df.to_json("filmy.json", orient="records")

## Další experimenty

In [30]:
import re

In [36]:
def kill_whitespace(x):
    def seriously(y):
        y = re.sub("\s+", " ", y)
        return(y)
    if isinstance(x, list):
        x = [seriously(y) for y in x]
    if isinstance(x, str):
        x = seriously(x)
    return(x)

In [37]:
df["Oceněníííí"] = 

In [39]:
df["Oceněníííí"].drop_duplicates().to_list()

[None,
 ['Vítěz Akce: Soutěž Filmové ligy československé na filmové libreto 1922 Praha / Československo I. cena za veselohruJarmila Hašková udělena v květnu 1922 v hodnoě 1 250 korun'],
 ['Vítěz Akce: Soutěž Filmové ligy československé na filmové libreto 1922 Praha / Československo I. cena za vážnou hruMarie Schäferová-Holešovská udělena v květnu 1922 v hodnoě 2 000 korun za libreto Případ Galginův'],
 ['Vítěz Akce: Zlatá medaile časopisu Filmový kurýr 1930 Praha / Československo Zlatá medaile Filmového kurýruMartin Frič udělena v březnu 1930 z rozhodnutí odborné poroty pro režiséra nejlepšího českého filmu roku 1929'],
 ['Vítěz Akce: Zlatá medaile časopisu Filmový kurýr 1930 Praha / Československo Označení jako nejlepší film roku 1929 konstatováno v březnu 1930 odbornou porotou, ale pro nevyhovění soutěžním podmínkám nemohl být klasifikován v soutěži o Zlatou medaili Filmového kurýru'],
 ['Vítěz Akce: Zlatá medaile časopisu Filmový kurýr 1930 Praha / Československo Zvláštní cenaJosef 

In [29]:
df[~df["Ocenění"].isnull()]["Ocenění"]

577     [Vítěz\n                                    Ak...
630     [Vítěz\n                                    Ak...
791     [Vítěz\n                                    Ak...
798     [Vítěz\n                                    Ak...
764     [Vítěz\n                                    Ak...
                              ...                        
4208    [Vítěz\n                                    Ak...
4167    [Vítěz\n                                    Fe...
4178    [Vítěz\n                                    Fe...
4209    [Vítěz\n                                    Fe...
4206    [Vítěz\n                                    Fe...
Name: Ocenění, Length: 1454, dtype: object

In [16]:
df["Poměr stran"].drop_duplicates().to_list()

[[1.33, 1.26],
 1.26,
 1.33,
 [1.33, 1.19],
 1.19,
 [1.37, 1.33],
 1.37,
 [1.37, 1.19],
 [1.85, 1.37],
 [1.66, 2.35],
 [1.37, 2.35],
 [1.66, 1.85],
 2.35,
 [1.85, 2.35],
 1.66,
 1.85,
 [1.66, 2.35, 2.55],
 [1.66, 1.37],
 None,
 [1.37, 1.78],
 2.55,
 2.39,
 1.78,
 [1.66, 1.37, 2.35],
 [1.85, 1.37, 1.19],
 [1.85, 1.78],
 [1.78, 1.33],
 [2.35, 1.78]]

In [17]:
df["Premiéra"].sample(60)

2165    premiéra 27. 8. 1970 /nepřístupný mládeži/ (ki...
2581             premiéra 23. 1. 1981 /přístupný mládeži/
511                                                  None
2449            premiéra 23. 12. 1977 /přístupný mládeži/
2405             premiéra 27. 5. 1977 /přístupný mládeži/
1183    premiéra 10. 11. 1939 /přístupný mládeži/ (kin...
3367                                                 None
3696    premiéra 21. 7. 2017 /nevhodné pro děti do 12 ...
2100    premiéra 8. 1. 1970 /přístupný mládeži/ (kino ...
1768    předpremiéra 15. 6. 1962  (kino Sevastopol /1 ...
2064    premiéra 29. 11. 1968 /přístupný mládeži/ (cel...
2993             premiéra 8. 12. 1991 /přístupný mládeži/
3835    festivalová premiéra 2. 7. 2018  (53. mezináro...
2890            premiéra 1. 9. 1989 /nepřístupný mládeži/
404     premiéra 8. 12. 2005 /přístupný/ (multiplex Vi...
4118    distribuční premiéra 9. 7. 2020 /nepřístupné p...
3638    festivalová premiéra 4. 7. 2015  (50. mezináro...
4523    premié

In [18]:
import re

In [19]:
def premiera(x, jaka):
    try:
        if f"{jaka}premiéra" in x:
            x = x.split(f"{jaka}premiéra")[1]
            try:
                datum = re.search('.*?\d{4}', x).group().replace(f"{jaka}premiéra","").strip()
                # datum = pd.to_datetime(datum, errors="coerce", format = "%d. %m. %Y")
            except:
                datum = ""
            try:
                misto = re.search('\((.*?)\)', x).group().replace("(","").replace(")","").strip()
            except AttributeError:
                misto = ""
        else:
            datum = ""
            misto = ""
    except TypeError:
        datum = ""
        misto = ""
    return [datum, misto]

In [20]:
df["Slavnostní premiéra (datum)"] = df["Premiéra"].apply(lambda x: premiera(x, "slavnostní ")[0])
df["Slavnostní premiéra (místo)"] = df["Premiéra"].apply(lambda x: premiera(x, "slavnostní ")[1])
df["Distribuční premiéra (datum)"] = df["Premiéra"].apply(lambda x: premiera(x, "distribuční ")[0])
df["Distribuční premiéra (místo)"] = df["Premiéra"].apply(lambda x: premiera(x, "distribuční ")[1])
df["Festivalová premiéra (datum)"] = df["Premiéra"].apply(lambda x: premiera(x, "festivalová ")[0])
df["Festivalová premiéra (místo)"] = df["Premiéra"].apply(lambda x: premiera(x, "festivalová ")[1])
df["Zahraniční premiéra (datum)"] = df["Premiéra"].apply(lambda x: premiera(x, "zahraniční ")[0])
df["Zahraniční premiéra (místo)"] = df["Premiéra"].apply(lambda x: premiera(x, "zahraniční ")[1])
df["Premiéra bez upřesnění (datum)"] = df["Premiéra"].apply(lambda x: premiera(x, "")[0])

In [21]:
df["Slavnostní premiéra (datum)"].nunique()

481

In [22]:
df.sort_values(by='Slavnostní premiéra (datum)',ascending=False)

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Přístupnost,Slavnostní premiéra (datum),Slavnostní premiéra (místo),Distribuční premiéra (datum),Distribuční premiéra (místo),Festivalová premiéra (datum),Festivalová premiéra (místo),Zahraniční premiéra (datum),Zahraniční premiéra (místo),Premiéra bez upřesnění (datum)
4115,Ženská pomsta (402078),"[Česká republika, Slovensko]",2020,2019—2020,"slavnostní premiéra 9. 9. 2020 (kino Lucerna,...",88.0,Dušan Rapoš (12892),film,komedie,"[hraný, distribuční]",...,None,9. 9. 2020,"kino Lucerna, Praha",10. 9. 2020,,,,,,9. 9. 2020
3983,Hodinářův učeň (401938),"[Česká republika, Slovensko]",2019,2018—2019,slavnostní premiéra 9. 8. 2019 (multiplex Cin...,104.0,Jitka Rudolfová (8110),film,"[komedie, pohádka]","[hraný, distribuční]",...,None,9. 8. 2019,"multiplex Cinema City Velký Špalíček, Brno",15. 8. 2019,"Bratislava, Slovensko",,,12. 9. 2019,"Bratislava, Slovensko",9. 8. 2019
4235,Bathory (40761),"[Slovensko, Česká republika, Velká Británie, M...",2008,2006,slavnostní premiéra 9. 7. 2008 (multiplex Pal...,138.0,"[české verze Petr Sitár (3697), Marie Fronková...",film,"[historický, thriller]","[hraný, distribuční]",...,None,9. 7. 2008,"multiplex Palace Cinemas Slovanský dům, Praha",10. 7. 2008,,,,,,9. 7. 2008
323,Správce statku (23974),Česká republika,2004,2001—2002,"slavnostní premiéra 9. 6. 2004 (kino Lucerna,...",78.0,Martin Duba (529),film,rodinný,"[hraný, distribuční]",...,None,9. 6. 2004,"kino Lucerna, Praha",10. 6. 2004,,,,,,9. 6. 2004
237,Nazareth – Nekonečný rockový mejdan (184216),Česká republika,2013,2011—2012,slavnostní premiéra 9. 5. 2013 (Praha) ...,90.0,Miloslav Šmídmajer (1129),film,hudební,"[dokumentární, distribuční]",...,None,9. 5. 2013,Praha,,,,,,,9. 5. 2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,Poslušně hlásím (396291),Československo,1957,1957,předpremiéra 27. 12. 1957 (kino Sevastopol /1...,92.0,Karel Steklý (35894),film,"[komedie, satira, válečný]","[hraný, distribuční]",...,None,,,,,,,,,27. 12. 1957
1586,Případ ještě nekončí (396290),Československo,1957,1957,předpremiéra 6. 9. 1957 (kino Sevastopol /1 t...,80.0,Ladislav Rychman (7892),film,detektivní,"[hraný, distribuční]",...,None,,,,,,,,,6. 9. 1957
1585,Přítel lhář (396289),Československo,1957,1957,premiéra 21. 6. 1957 /nepřístupný mládeži/ (ki...,30.0,Jindřich Puš (83596),film,příběh,"[hraný, distribuční]",...,None,,,,,,,,,21. 6. 1957
1584,Ročník 21 (396288),"[Československo, Německá demokratická republika]",1957,1957,premiéra 21. 2. 1958 /nepřístupný mládeži/ (ki...,95.0,Václav Gajer (5065),film,drama,"[hraný, distribuční]",...,None,,,,,,,,,21. 2. 1958


In [23]:
df[df["Copyright"] == 1898].sort_values(by="Premiéra bez upřesnění (datum)")

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Přístupnost,Slavnostní premiéra (datum),Slavnostní premiéra (místo),Distribuční premiéra (datum),Distribuční premiéra (místo),Festivalová premiéra (datum),Festivalová premiéra (místo),Zahraniční premiéra (datum),Zahraniční premiéra (místo),Premiéra bez upřesnění (datum)
3767,Útok pražského dělostřelectva (401716),Rakousko-Uhersko,1898,1898,premiéra 07/1898 (Výstava architektury a inže...,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,07/1898
412,Smích a pláč (395110),Rakousko-Uhersko,1898,1898,premiéra 07/1898 (Výstava architektury a inže...,1.0,Jan Kříženecký (127419),film,anekdota,"[hraný, distribuční]",...,None,,,,,,,,,07/1898
3764,Rychlovlak v Podbabí (401713),Rakousko-Uhersko,1898,1898,premiéra 07/1898 (Výstava architektury a inže...,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,07/1898
3762,Přenesení kolébky Františka Palackého z Hodsla...,Rakousko-Uhersko,1898,1898,premiéra 07/1898 (Výstava architektury a inže...,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,07/1898
3766,Svatojanská pouť v českoslovanské vesnici (401...,Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a ...,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,19. 6. 1898
3763,Purkyňovo náměstí na Královských Vinohradech (...,Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a ...,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,19. 6. 1898
3761,Polední výstřel na Mariánských hradbách (401709),Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a ...,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,19. 6. 1898
3758,Defilování vojska o Božím těle na Královských ...,Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a ...,1.0,Jan Kříženecký (127419),film,reportáž,"[dokumentární, distribuční]",...,None,,,,,,,,,19. 6. 1898
413,Výstavní párkař a lepič plakátů (395111),Rakousko-Uhersko,1898,1898,premiéra 24. 7. 1898 (Výstava architektury a ...,1.0,Jan Kříženecký (127419),film,fraška,"[hraný, distribuční]",...,None,,,,,,,,,24. 7. 1898
3768,Voltýžování jízdního odboru Sokola pražského (...,Rakousko-Uhersko,1898,1898,premiéra 24. 7. 1898 (Výstava architektury a ...,1.0,Jan Kříženecký (127419),film,"[reportáž, sportovní]","[dokumentární, distribuční]",...,None,,,,,,,,,24. 7. 1898


In [24]:
df.groupby(df['Premiéra bez upřesnění (datum)'].dt.weekday)["Film"].nunique().nlargest(50)

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
df.groupby("Festivalová premiéra (místo)")["Film"].nunique().nlargest(50)

In [ ]:
df.groupby("Copyright")["Slavnostní premiéra (místo)"].nunique().plot()

In [ ]:
top_kina = df.groupby("Slavnostní premiéra (místo)")["Film"].nunique().nlargest(10).dropna().index.to_list()

In [ ]:
top_kina = [x for x in top_kina if x != ""]

In [ ]:
df[(df["Slavnostní premiéra (místo)"].isin(top_kina)) & (df["Copyright"] > 1989)].groupby(["Copyright","Slavnostní premiéra (místo)"])["Slavnostní premiéra (datum)"].nunique().unstack().plot()

In [ ]:
kino_mask = df['Slavnostní premiéra (místo)'].str.contains('kino')
multiplex_mask = df['Slavnostní premiéra (místo)'].str.contains('multiplex')

In [ ]:
import numpy as np

In [ ]:
df['group'] = pd.Series(
    np.where(kino_mask, 'kino', None) +
    np.where(multiplex_mask, 'multiplex', None)
)

In [ ]:
df.groupby(["Copyright", df["Slavnostní premiéra (místo)"].str.contains("kino|multiplex")])["Slavnostní premiéra (datum)"].nunique().plot()

In [ ]:
df["Lokace"].to_list()